# LLMs for Input Classification & Structured Output

This notebook demonstrates how to use an LLM for a common and powerful task: **classifying user input**. We instruct the model to categorize customer service queries into a predefined two-level hierarchy.

### Key Techniques Shown:

* **Classification via Prompting**: The `system` message provides the model with detailed instructions, including the list of valid primary and secondary categories to choose from.
* **Guaranteed JSON Output**: We use `response_format={"type": "json_object"}` in the API call. This is a crucial feature that forces the model's output to be a syntactically correct JSON object, making it reliable for use in downstream applications.
* **Clear Input Separation**: Delimiters (`####`) are used to cleanly separate the raw user query from the instructions in the prompt, which is a best practice for improving model performance.

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [4]:
openai.ChatCompletion.create?

In [6]:
def get_completion_from_messages(messages, 
                                 model="gpt-4o", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
        response_format={"type": "json_object"} 
    )
    return response.choices[0].message["content"]

#### Classify customer queries to handle different cases

In [7]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Classify each query into a primary category \
and a secondary category. 
Provide your output in json format with the \
keys: primary and secondary.

Primary categories: Billing, Technical Support, \
Account Management, or General Inquiry.

Billing secondary categories:
Unsubscribe or upgrade
Add a payment method
Explanation for charge
Dispute a charge

Technical Support secondary categories:
General troubleshooting
Device compatibility
Software updates

Account Management secondary categories:
Password reset
Update personal information
Close account
Account security

General Inquiry secondary categories:
Product information
Pricing
Feedback
Speak to a human

"""
user_message = f"""\
I want you to delete my profile and all of my user data"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

{
  "primary": "Account Management",
  "secondary": "Close account"
}


In [8]:
user_message = f"""\
Tell me more about your flat screen tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

{
  "primary": "General Inquiry",
  "secondary": "Product information"
}


In [9]:
# The list of queries to test
customer_queries = [
    "I want to cancel my subscription.",
    "My app is crashing on my new Android phone. Can you help?",
    "How much does your premium plan cost?",
    "I was charged twice this month, please refund one of the charges.",
    "I can't remember my password and need to reset it.",
    "I need to speak to a human agent right away.",
    "How can I update my credit card information?",
    "I think someone else accessed my account, what should I do?",
    "Just wanted to say your new update is fantastic!",
    "What are the main differences between the free and pro versions?"
]


# Loop through each query and print the classification
for query in customer_queries:
    print(f"Query: {query}")
    
    messages =  [  
        {'role':'system', 
         'content': system_message},    
        {'role':'user', 
         'content': f"{delimiter}{query}{delimiter}"},  
    ] 
    
    response = get_completion_from_messages(messages)
    print(f"Response: {response}\n" + "="*100)

Query: I want to cancel my subscription.
Response: {
  "primary": "Billing",
  "secondary": "Unsubscribe or upgrade"
}
Query: My app is crashing on my new Android phone. Can you help?
Response: {
  "primary": "Technical Support",
  "secondary": "Device compatibility"
}
Query: How much does your premium plan cost?
Response: {
  "primary": "General Inquiry",
  "secondary": "Pricing"
}
Query: I was charged twice this month, please refund one of the charges.
Response: {
  "primary": "Billing",
  "secondary": "Dispute a charge"
}
Query: I can't remember my password and need to reset it.
Response: {
  "primary": "Account Management",
  "secondary": "Password reset"
}
Query: I need to speak to a human agent right away.
Response: {
  "primary": "General Inquiry",
  "secondary": "Speak to a human"
}
Query: How can I update my credit card information?
Response: {
  "primary": "Billing",
  "secondary": "Add a payment method"
}
Query: I think someone else accessed my account, what should I do?
Res